In [7]:
import requests
import pandas as pd
import logging

from typing import Union
from functions import format_timedelta

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

from constants import FEAR_GREED_INDEX_URL
URL = FEAR_GREED_INDEX_URL


def get_index(
    url: str,
    timeout: int = 10,
    limit: int = 10,
    format: str = "json"
) -> Union[dict, None]:
    """
    Fetches the Fear & Greed Index data from the specified URL.

    Parameters:
    - url (str): The API endpoint URL with placeholders for limit and format.
    - timeout (int): The timeout for the HTTP request in seconds.
    - limit (int): The number of data points to retrieve.
    - format (str): The response format, either 'json' or 'csv'.

    Returns:
    - dict or None: The JSON response as a dictionary if successful, otherwise None.
    """
    url = url.format(limit=limit, format=format)
    try:
        response = requests.get(url, timeout=timeout).json()
        raw = response['data']
        df = pd.DataFrame(raw)
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        df['value_classification'] = df['value_classification'].astype('str')
        df['timestamp'] = pd.to_numeric(df['timestamp'])
        df['date'] = pd.to_datetime(df['timestamp'], unit='s')
        df.sort_values(by = 'date', ascending = False)
        df['time_until_update'] = pd.to_numeric(df['time_until_update'])
        df['time_until_update'] = pd.to_timedelta(df['time_until_update'], unit='s')
        df['time_until_update'] = format_timedelta(df['time_until_update'])
        df.drop(columns = 'timestamp', inplace = True)
        return df
    except requests.exceptions.RequestException as e:
        logger.error("Error getting Fear & Greed data: %s", e)
        return None

In [8]:
fg = get_index(URL, timeout = 10, limit=100, format = 'json')
fg

/home/geraltwolf/CryptoDataAnalysis/crypto_dashboard/functions.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3 hours and 32 minutes' has dtype incompatible with timedelta64[ns], please explicitly cast to a compatible dtype first.
  res.iloc[0] = formatted


,value,value_classification,time_until_update,date
0,30,Fear,3 hours and 32 minutes,2025-10-24
1,27,Fear,NaT,2025-10-23
2,25,Extreme Fear,NaT,2025-10-22
3,34,Fear,NaT,2025-10-21
4,29,Fear,NaT,2025-10-20
...,...,...,...,...
95,71,Greed,NaT,2025-07-21
96,72,Greed,NaT,2025-07-20
97,74,Greed,NaT,2025-07-19
98,73,Greed,NaT,2025-07-18
